In [20]:
import torch
from torch.fft import fft2, ifft2, fftshift, ifftshift
import torchvision
import numpy as np

import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm
from dps.guided_diffusion.unet import create_model

from ddrm_jpeg.datasets.__init__ import get_dataset, data_transform, inverse_data_transform
from ddrm_jpeg.functions.ckpt_util import get_ckpt_path, download
from ddrm_jpeg.functions.denoising import jpeg_steps
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Load model
model_config = {'image_size': 256,
                'num_channels': 128,
                'num_res_blocks': 1,
                'channel_mult': '',
                'learn_sigma': True,
                'class_cond': False,
                'use_checkpoint': False,
                'attention_resolutions': 16,
                'num_heads': 4,
                'num_head_channels': 64,
                'num_heads_upsample': -1,
                'use_scale_shift_norm': True,
                'dropout': 0.0,
                'resblock_updown': True,
                'use_fp16': False,
                'use_new_attention_order': False,
                'model_path': 'ffhq_10m.pt'}
model = create_model(**model_config)
model = model.to(device)
# use in eval mode:
model.eval();